In [1]:
import pandas as pd
from os import listdir
import numpy as np

In [2]:
directory = "C:/Users/ricab/Desktop/data-x/Project/ead-model/data/ados_datasets/"

In [7]:
filename='current_data.txt'
txtname = directory+filename
data = pd.read_csv(txtname, sep="\t", header=0)

In [8]:
data

,Unnamed: 0,ados diagnosis classification,age in months at the time of the interview/test/sampling/imaging.,anxiety,collection_id,collection_title,dataset_id,date on which the interview/genetic test/sampling/imaging/biospecimen was completed. mm/dd/yyyy,hand and finger and other complex mannerisms,imagination/creativity,...,overall ados diagnosis,promoted_subjectkey,quality of social overtures,self-injurious behavior,sex of the subject,shared enjoyment in interaction,subject id how it's defined in lab/project,"tantrums, aggression, negative or disruptive behavior",the ndar global unique identifier (guid) for research subject,unusual eye contact
0,0,Non-Spectrum,35,0.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,05/21/2012,0.0,0.0,...,Type 1,NDARAC381MK6,0.0,0.0,F,0.0,U3H5S,0.0,NDARAC381MK6,0.0
1,1,non-spectrum,30,0.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,01/16/2009,0.0,2.0,...,"non-spectrum, late walker dx gross motor delay...",NDARAD431MTD,0.0,0.0,F,0.0,P3U3P,0.0,NDARAD431MTD,0.0
2,2,non-spectrum,23,1.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,06/03/2009,0.0,1.0,...,control,NDARAF706DLN,0.0,0.0,M,0.0,P6C6P,0.0,NDARAF706DLN,0.0
3,3,non-spectrum,31,0.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,01/28/2010,0.0,0.0,...,typical control,NDARAF706DLN,0.0,0.0,M,0.0,P6C6P,0.0,NDARAF706DLN,0.0
4,4,non-spectrum,13,1.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,05/01/2009,1.0,1.0,...,AD,NDARAF772NJQ,1.0,1.0,F,1.0,S4L5J,1.0,NDARAF772NJQ,0.0
5,5,ASD,35,1.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,03/28/2011,1.0,2.0,...,AD/PDD-NOS,NDARAJ262WZJ,1.0,0.0,M,2.0,D5U7E,2.0,NDARAJ262WZJ,0.0
6,6,ASD,31,0.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,11/04/2010,0.0,2.0,...,AD,NDARAJ898KYP,2.0,0.0,M,1.0,L5S3Z,0.0,NDARAJ898KYP,2.0
7,7,Non-spectrum,28,0.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,02/28/2009,0.0,2.0,...,Typical,NDARAL181JRQ,0.0,0.0,M,0.0,R3E9X,0.0,NDARAL181JRQ,0.0
8,8,ASD,26,1.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,09/10/2008,2.0,2.0,...,PDD-NOS,NDARAL517BU0,1.0,0.0,M,0.0,N8T8F,0.0,NDARAL517BU0,0.0
9,9,ASD,41,0.0,9,Biomarkers of Autism at 12 Months: From Brain ...,8161,11/19/2010,2.0,3.0,...,AD,NDARAM797KX6,2.0,0.0,M,1.0,P2Z4H,1.0,NDARAM797KX6,2.0


In [15]:
data=data.drop(['Unnamed: 0', 'collection_id', 'collection_title', 'dataset_id', 'promoted_subjectkey', 'the ndar global unique identifier (guid) for research subject'], axis=1)

In [16]:
data

,ados diagnosis classification,age in months at the time of the interview/test/sampling/imaging.,anxiety,date on which the interview/genetic test/sampling/imaging/biospecimen was completed. mm/dd/yyyy,hand and finger and other complex mannerisms,imagination/creativity,immediate echolalia,overall ados diagnosis,quality of social overtures,self-injurious behavior,sex of the subject,shared enjoyment in interaction,subject id how it's defined in lab/project,"tantrums, aggression, negative or disruptive behavior",unusual eye contact
0,Non-Spectrum,35,0.0,05/21/2012,0.0,0.0,0.0,Type 1,0.0,0.0,F,0.0,U3H5S,0.0,0.0
1,non-spectrum,30,0.0,01/16/2009,0.0,2.0,1.0,"non-spectrum, late walker dx gross motor delay...",0.0,0.0,F,0.0,P3U3P,0.0,0.0
2,non-spectrum,23,1.0,06/03/2009,0.0,1.0,0.0,control,0.0,0.0,M,0.0,P6C6P,0.0,0.0
3,non-spectrum,31,0.0,01/28/2010,0.0,0.0,0.0,typical control,0.0,0.0,M,0.0,P6C6P,0.0,0.0
4,non-spectrum,13,1.0,05/01/2009,1.0,1.0,2.0,AD,1.0,1.0,F,1.0,S4L5J,1.0,0.0
5,ASD,35,1.0,03/28/2011,1.0,2.0,2.0,AD/PDD-NOS,1.0,0.0,M,2.0,D5U7E,2.0,0.0
6,ASD,31,0.0,11/04/2010,0.0,2.0,1.0,AD,2.0,0.0,M,1.0,L5S3Z,0.0,2.0
7,Non-spectrum,28,0.0,02/28/2009,0.0,2.0,0.0,Typical,0.0,0.0,M,0.0,R3E9X,0.0,0.0
8,ASD,26,1.0,09/10/2008,2.0,2.0,8.0,PDD-NOS,1.0,0.0,M,0.0,N8T8F,0.0,0.0
9,ASD,41,0.0,11/19/2010,2.0,3.0,1.0,AD,2.0,0.0,M,1.0,P2Z4H,1.0,2.0


In [30]:
data1=data[(data['ados diagnosis classification']=='ASD') | (data['ados diagnosis classification']=='non-spectrum')]

In [33]:
data1[['ados diagnosis classification', 'imagination/creativity']]

,ados diagnosis classification,imagination/creativity
1,non-spectrum,2.0
2,non-spectrum,1.0
3,non-spectrum,0.0
4,non-spectrum,1.0
5,ASD,2.0
6,ASD,2.0
8,ASD,2.0
9,ASD,3.0
10,ASD,1.0
11,non-spectrum,2.0
